# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import matplotlib.pyplot as plt
from matplotlib import style

2022-11-24 17:52:04.120263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 17:52:04.374483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 17:52:04.374501: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 17:52:04.410617: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-24 17:52:05.100261: W tensorflow/stream_executor/platform/de

In [2]:
# Dataset Train preprocesado
train_ds = pd.read_csv('datasets/tp1-train.csv')
train_ds = train_ds.drop(['Unnamed: 0'], axis=1)
train_ds.head()

,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,2021-07-29,2021-09-06 00:00:00.000000000,-34.564800,-58.444631,Belgrano,Departamento,5.0,4.0,279.0,245.0,1000000.0,Espectacular torre en la mejor zona de Belgran...
1,2021-12-16,2022-02-09 00:00:00.000000000,-34.591359,-58.418809,Palermo,Departamento,2.0,1.0,42.0,42.0,89900.0,VENTA 2 AMBIENTES RECICLADO - PALERMO
2,2021-06-19,2021-09-01 15:14:13.531306906,-34.602632,-58.428399,Almagro,Departamento,2.0,1.0,46.0,42.0,119900.0,2 ambientes al frente con balcón. A estrenar.
3,2021-12-16,2022-02-09 00:00:00.000000000,-34.564290,-58.439272,Palermo,Departamento,3.0,2.0,69.0,58.0,175000.0,DEPARTAMENTO 3 Ambientes Palermo-Balcón C/Parr...
4,2021-12-16,2022-02-09 00:00:00.000000000,-34.630517,-58.377143,Barracas,Departamento,1.0,1.0,45.0,41.0,120000.0,Venta-Barracas-Mono divisible- MOCA


In [3]:
# Dataset Test preprocesado
test_ds = pd.read_csv('datasets/tp1-test.csv')
test_ds = test_ds.drop(['Unnamed: 0'], axis=1)
test_ds

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,-34.589649,-58.434926,Palermo,Casa,4.0,3.0,140.0,140.0,530000.0,Casa - Palermo
1,-34.598508,-58.443084,Villa Crespo,Departamento,1.0,1.0,33.0,30.0,78000.0,Monoambiente Departamento a Estrenar - Villa C...
2,-34.582098,-58.448859,Palermo,Departamento,1.0,1.0,35.0,30.0,87100.0,Venta - Newbery Jorge 3445 3º I Jardin
3,-34.569662,-58.487762,Villa Urquiza,Departamento,3.0,2.0,80.0,68.0,250000.0,Departamento - Villa Urquiza
4,-34.598582,-58.376689,San Nicolás,Departamento,2.0,1.0,37.0,37.0,79000.0,Venta de departamento estilo frances de 2 amb...
...,...,...,...,...,...,...,...,...,...,...
16271,-34.652357,-58.385489,Barracas,Casa,5.0,4.0,223.0,223.0,295000.0,CASA EN LOTE DE 10X27 MTS 223 M2 CUB - RIO CUA...
16272,-34.653920,-58.410842,Pompeya,Casa,4.0,2.0,257.0,130.0,180000.0,VENTA-POMPEYA-CASA-2PLANTAS-DIST TECNOLOGICO
16273,-34.593420,-58.421077,Palermo,Departamento,3.0,2.0,91.0,71.0,348000.0,"Excelente oportunidad, Hermoso 3 Ambientes, nu..."
16274,-34.593404,-58.402309,Barrio Norte,Departamento,2.0,1.0,45.0,45.0,133000.0,Departamento - Barrio Norte


## Procesamiento del lenguaje natural

### Ampliación del dataset

Este dataset incluye descripciones de las propiedades del otro dataset. Veremos como podemos extraer información de estas descripciones.

In [4]:
descriptions_dataset = pd.read_csv('datasets/properati_argentina_2021_decrip.csv')

In [5]:
descriptions_dataset

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
...,...,...
460149,MbQ4rqEzjp9Jfc3Sywb/YQ==,Corredor Responsable: Hector Garaygorta / Cris...
460150,fbuG1ohYC623nxLYPQPBvw==,Corredor Responsable: MARIA DEL ROSARIO NEME -...
460151,Ry5hMwctFXD7rLoUpzduxQ==,En cumplimiento de las leyes provinciales vige...
460152,xH37f+l+VOP8Vj0T3g+r6g==,En cumplimiento de las leyes provinciales vige...


#### Tecnica Minqing Hu y Bing Liu

Una forma de analizar el sentimiento de un de un texto es considerando su sentimiento como la suma de los sentimientos de cada una de las palabras que lo forman.

Para el analisis de sentimiento nos guiamos del analisis realizado en esta pagina: https://www.cienciadedatos.net/documentos/py25-text-mining-python.html

Utilizamos algunas funciones de tokenizacion y limpieza de ahi con alguna sutil modificacion para nuestro caso de uso en particular.

In [6]:
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = str(texto).lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\¡\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    
    return(nuevo_texto)

In [7]:
# se aplica la función de limpieza y tokenización a cada descripcion

descriptions_dataset['texto_tokenizado'] = descriptions_dataset['property_description'].apply(limpiar_tokenizar)
descriptions_dataset[['texto_tokenizado']].head()

,texto_tokenizado
0,"[corredor, responsable, maria, alejandra, geno..."
1,"[hermoso, ph, tipo, casa, de, ambientes, br, a..."
2,"[excelente, ph, en, núñez, amb, oportunidad, t..."
3,"[unico, ph, de, dormitorios, en, impecable, es..."
4,"[ph, amb, villa, crespo, exclusivo, uso, profe..."


In [8]:
# unnest de la columna texto_tokenizado

descriptions_tidy = descriptions_dataset.explode(column='texto_tokenizado')
descriptions_tidy = descriptions_tidy.drop(columns='property_description')
descriptions_tidy = descriptions_tidy.rename(columns={'texto_tokenizado':'token'})
descriptions_tidy.head(3)

,id,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable
0,BZCeiGkJr1WBUi6pKJQwJQ==,maria


Notamos que las preposiciones no son relevantes para entender que atributo podria ser mejor para expandir el datast, asi que decidimos agregarlas como stopwords.

Tampoco van a variar mucho el analisis de sentimiento realizado en este trabajo.

In [9]:
## listado de stopwords

preposiciones = ["a", "ante", "bajo", "(cabe)", "con", "contra", "de", "desde", "durante", "en", "entre", "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "vía"]

stop_words = []

stop_words += preposiciones

# filtrado para excluir stopwords
descriptions_tidy = descriptions_tidy[~(descriptions_tidy["token"].isin(stop_words))]

Agregamos un lexicon en español de esta pagina: https://github.com/jboscomendoza/lexicos-nrc-afinn

In [10]:
# lexicon sentimientos
lexicon = pd.read_csv('datasets/lexico_nrc.csv')
lexicon

,palabra,sentimiento,word
0,ciegamente,negativo,blindly
1,ciegamente,tristeza,blindly
2,a distancia,negativo,aloof
3,a hurtadillas,sorpresa,stealthily
4,a la deriva,anticipación,adrift
...,...,...,...
15302,zumbido,anticipación,buzz
15303,zumbido,miedo,buzz
15304,zumbido,positivo,buzz
15305,zumbido,negativo,drone


In [11]:
def mappear_valores_sentimiento(s):
    # 1 Positivo
    # 0 Neutro
    # -1 Negativo
    sentimiento_numerico = 0
    if str(s) in ['negativo', 'tristeza', 'miedo', 'enfado', 'tristeza', 'asco']:
        sentimiento_numerico = -1
    if str(s) in ['sorpresa', 'positivo', 'confianza', 'alegría']:
        sentimiento_numerico = 1
    if str(s) in ['anticipación']:
        sentimiento_numerico = 0
        
    return sentimiento_numerico
    

In [12]:
lexicon['sentimiento'] = lexicon['sentimiento'].apply(lambda x: mappear_valores_sentimiento(x))
lexicon[['sentimiento']].head()

,sentimiento
0,-1
1,-1
2,-1
3,1
4,0


In [13]:
descriptions_tidy[~descriptions_tidy.token.isin(lexicon.palabra)].head()

,id,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,maria
0,BZCeiGkJr1WBUi6pKJQwJQ==,alejandra
0,BZCeiGkJr1WBUi6pKJQwJQ==,genoveva
0,BZCeiGkJr1WBUi6pKJQwJQ==,bohn


In [14]:
lexicon[~lexicon.palabra.isin(descriptions_tidy.token)].head()

,palabra,sentimiento,word
0,ciegamente,-1,blindly
1,ciegamente,-1,blindly
2,a distancia,-1,aloof
3,a hurtadillas,1,stealthily
4,a la deriva,0,adrift


In [15]:
# sentimiento promedio de cada descripcion
descripcion_sentimientos = pd.merge(
                            left     = descriptions_tidy,
                            right    = lexicon,
                            left_on  = "token", 
                            right_on = "palabra",
                            how      = "inner"
                      )

descripcion_sentimientos = descripcion_sentimientos.drop(columns = ["palabra", "word"])
descripcion_sentimientos

,id,token,sentimiento
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
1,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
2,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
3,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
4,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
...,...,...,...
17186156,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186157,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186158,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186159,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1


In [16]:
descripcion_sentimientos

,id,token,sentimiento
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
1,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
2,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
3,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
4,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
...,...,...,...
17186156,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186157,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186158,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186159,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1


In [18]:
propiedades_sentimientos = descripcion_sentimientos[["id", "token", "sentimiento"]] \
                      .groupby(["id"])\
                      .sum().reset_index()

In [19]:
propiedades_sentimientos

,id,sentimiento
0,+++LrfX9dl/rsnbNTgVaSw==,24
1,+++do60YqD8lHYx2VYGDqQ==,1
2,+++wa5fxPpZiosDy8gUAAw==,2
3,+++z1sn9bNyplgPCYXZlyw==,7
4,++/5cFjr0gDqo7tRQJ+crA==,10
...,...,...
452586,zzxdSRjo9Dmr+YRiF12AKw==,2
452587,zzxkD9EywKS2qVHckCrB6w==,19
452588,zzz31+5xq/hpoj0wVg95MQ==,2
452589,zzz3xbg/jr1o6a3FBduweQ==,-2


In [18]:
propiedades_sentimientos.sentimiento.max()

273

In [24]:
top10_positivas = propiedades_sentimientos[propiedades_sentimientos['sentimiento'] > 248]

In [27]:
descriptions_dataset[descriptions_dataset.id.isin(top10_positivas.id)].property_description

55559     VENTA Depto 303 módulo 3 - 156 m2 AKOL BARRIO ...
112373    VENTA Casa de 282m2 de excelente calidad sobre...
123306    VENTA Depto AKOL BARRIO NÁUTICO, Barrio Tres C...
249077    VENTA Depto 302 módulo 3 - 129 m2 AKOL BARRIO ...
311142    VENTA Casa 4 dorm. 3 baños, 218 m2 - Lote 14 L...
311189    Corredor Responsable: Cristian Arnal Ponti - C...
312277    VENTA Departamento 305 135m2 Mod3 AKOL BARRIO ...
312278    VENTA Departamento 306 Módulo 3 - 172 m2 AKOL ...
312279    AKOL PATAGONIA BARRIO NÁUTICO<br>LANZAMIENTO M...
391337    CARACTERÍSTICAS: Lote en inmejorable ubicación...
Name: property_description, dtype: object

In [28]:
propiedades_sentimientos.sentimiento.min()

-71

In [29]:
top10_negativas = propiedades_sentimientos[propiedades_sentimientos['sentimiento'] < -33]

In [31]:
descriptions_dataset[descriptions_dataset.id.isin(top10_negativas.id)].property_description

67053     Superficie Terreno:   \t7 hectáreas (69.625 me...
104195    Corredor Responsable: JUAN FRANCISCO CARATTINO...
118082    Propiedad en dos plantas edificada sobre la lí...
194147    Propiedad en dos plantas edificada sobre la lí...
213430    ESPECTACULAR   E  IMPECABLE !!!!!<br>REFACCION...
227431    HERMOSO COMPLEJO DE  18 UNIDADES FUNCIONALES, ...
242409    Espectacular casa en Pilar Golf barrio Pevero<...
267009    VENTA DE CASA EN LA ARBOLEDA.<br>Terreno de 20...
299463    Espectacular casa en Pilar Golf barrio Pevero<...
Name: property_description, dtype: object

In [24]:
## A partir de tener el top10, evaluar si esos atributos nos sirven para expandir el dataset
## Pensar en hacer un score con la completitud de la descripcion de cada propiedad -> property_desc_score

Como es claro, la gente que publica la venta de una propiedad va a tratar de expresar la mejor publicacion y descripcion posible. Es por eso que tenemos una tasa altisima de positividad. No buscamos hacer un analisis real de a

In [32]:
def perfil_sentimientos(df):
    print(f"Positivos: {round(100 * np.mean(df.sentimiento > 0), 2)}")
    print(f"Neutros  : {round(100 * np.mean(df.sentimiento == 0), 2)}")
    print(f"Negativos: {round(100 * np.mean(df.sentimiento < 0), 2)}")
    print(" ")

perfil_sentimientos(descripcion_sentimientos)

Positivos: 64.61
Neutros  : 9.64
Negativos: 25.75
 


#### Tecnica Regex

Revisamos las siguientes paginas para entender cuales son los ammenities mas buscados en CABA, y en Argentina en general.

https://www.iprofesional.com/negocios/371702-cuales-son-los-amenities-mas-exoticos-de-edificios-en-argentina

https://www.baenegocios.com/sociedad/Ranking-de-amenities-los-servicios-que-mas-pesan-al-comprar-una-propiedad-20220119-0068.html

https://www.forbesargentina.com/negocios/amenities-servicios-mas-demandados-argentinos-comprar-una-propiedad-n11901

Dichos ammenities parecen hacer que la propiedad cotice entre un 15% y un 20% más que el precio de venta.

Sacando un promedio y haciendo un top-5 ranking, podemos notar que los mas relevantes son:

- Garage/Estacionamiento
- Pileta
- Jardin/Espacio al aire libre
- Parrilla
- SUM (Gimnasio/Spa/Sauna)

Al buscar estos datos, podriamos tratar de entender si el precio resulta mayor, contra una propiedad de similares caracteristicas pero sin estos ammenities y a partir de eso, entender que % varía del precio de venta original.

In [ ]:
garage_sin_espacio = re.compile(r"[^\s]+garage|garaje|estacionamiento|parking")
garage_con_espacio = re.compile(r"[^\s]+ garage|garaje|estacionamiento|parking")

In [ ]:
frecuencia_garage = {'garage': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(garage_sin_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
    if (len(re.findall(garage_con_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
print(frecuencia_garage)

In [ ]:
pileta_sin_espacio = re.compile(r"[^\s]+pileta")
pileta_con_espacio = re.compile(r"[^\s]+ pileta")

In [ ]:
frecuencia_pileta= {'pileta': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(pileta_sin_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
    if (len(re.findall(pileta_con_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
print(frecuencia_pileta)

In [ ]:
jardin_sin_espacio = re.compile(r"[^\s]+jardin|espacio verde")
jardin_con_espacio = re.compile(r"[^\s]+ jardin|espacio verde")

In [ ]:
frecuencia_jardin = {'jardin': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(jardin_sin_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
    if (len(re.findall(jardin_con_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
print(frecuencia_jardin)

In [ ]:
parrilla_sin_espacio = re.compile(r"[^\s]+parrilla|bbq")
parrilla_con_espacio = re.compile(r"[^\s]+ parrilla|bbq")

In [ ]:
frecuencia_parrilla = {'parrilla': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(parrilla_sin_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
    if (len(re.findall(parrilla_con_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
print(frecuencia_parrilla)

In [ ]:
zoom_sin_espacio = re.compile(r"[^\s]+zoom|sum|gimansio|spa")
zoom_con_espacio = re.compile(r"[^\s]+ zoom|sum|gimansio|spa")

In [ ]:
frecuencia_zoom = {'sum': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(zoom_sin_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
    if (len(re.findall(zoom_con_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
print(frecuencia_zoom)

Notamos que muchas obervaciones tenian el dato de si contaban o no con balcon asi que decidimos agregarlo al analisis para poder ver si encontramos una correlacion que nos de mas informacion con lo que respecta al balcon y a la superifice total versus la superificie cubierta y el precio de venta de cada vivienda.

In [ ]:
balcon_sin_espacio = re.compile(r"[^\s]+balcon")
balcon_con_espacio = re.compile(r"[^\s]+ balcon")

In [ ]:
frecuencia_balcon= {'balcon': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(balcon_sin_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
    if (len(re.findall(balcon_con_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
print(frecuencia_balcon)

Otro factor interesante que podriamos analizar es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una ammenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta.
De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

#### Tecnica Open Information Extraction

#### Modelos

## Redes Neuronales

### Regresión

### Regresión

### Clasificación

#### Construcción del target

#### Modelo

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones